In [25]:
import sys
import os

from Networks.MLP import MLP
from Networks.CNN import CNN

import matplotlib.pyplot as plt
from dataset.mnist import load_mnist
from Layers.optimizer import *
from Layers.util import smooth_curve
import torch
import pandas as pd

try:
    import cupy as np
    py = 'cupy'
except ImportError:
    import numpy as np
    py = 'numpy'
import time

In [26]:
def test(test_list, data_name, max_acc=None):
    tr_data, tr_lb, ts_data, ts_lb = set_data(data_name)
    tr_size = tr_data.shape[0]
    ts_size = ts_data.shape[0]
    
    networks = {}
    train_acc = {}
    train_loss = {}
    test_acc = {}
    test_loss = {}
    optimizer = {}
    total_step = {}
    time_record = {}
    print(f'[Training]\n{py}\n')
    print(f'[Shape]\ntrain data: {tr_data.shape}\ntrain label: {tr_lb.shape}\ntest data: {ts_data.shape}\ntest label: {ts_lb.shape}')
    
    for key in test_list.keys():
        # 기록 리스트 생성
        train_acc[key] = []
        train_loss[key] = []
        test_acc[key] = []
        test_loss[key] = []
        total_step[key] = 0
        time_record[key] = 0

        # nSigmoid 경우 파라미터
        # key == 'nSigmoid_8' -> act = nSigmoid, threshold = 8
        networks[key] = test_list[key]['network']

        # optimizer 생성
        opt = test_list[key]['opt']
        lr = test_list[key]['lr']
        if opt == 'SGD':
            optimizer[key] = SGD(lr)
        elif opt == 'Momentum':
            optimizer[key] = Momentum(lr)
        elif opt == 'AdaGrad':
            optimizer[key] = AdaGrad(lr)
        elif opt == 'Adam':
            optimizer[key] = Adam(lr)
        else:
            assert 'no opt' + opt

    # 파라미터 통일 셋팅
    # networks1 = networks['1']
    # networks2 = networks['2']
    # for k in networks1.params.keys():
    #     for i in range(networks1.params[k].shape[0]):
    #         networks1.params[k][i] = networks2.params[k][i].copy()
    # print(np.max(networks['1'].params['W1'] - networks['2'].params['W1']))
    # 학습
    for i in range(max_iterations):
        if 0 not in total_step.values():
            print("All test list training Done\nStep: " + str(i))
            break

        # mini-batch train
        batch_mask = np.random.choice(tr_size, batch_size)
        tr_data_b = tr_data[batch_mask]
        tr_lb_b = tr_lb[batch_mask]

        # mini-batch test
        batch_mask_t = np.random.choice(ts_size, batch_size)
        ts_data_b = ts_data[batch_mask_t]
        ts_lb_b = ts_lb[batch_mask_t]

        # print_iter 회마다 경과 출력
        if i % print_iter == 0:
            print( "="*15 + "iteration:" + str(i) + "="*15)
            print("{:^9}|{:^9}|{:^9}|{:^9}".format('model','time','acc','loss'))

        # 학습 & 추론 & 기록
        for key in test_list.keys():
            if total_step[key] != 0:
                # Max acc 에 도달해 학습이 끝난 test model
                continue
            else:
                start = time.time()
                # CV 모델 데이터 처리
                if isinstance(networks[key], CNN) and tr_data.ndim == 2:
                    tr_data_b = tr_data_b.reshape(-1, 1, 28, 28)
                    ts_data_b = ts_data_b.reshape(-1, 1, 28, 28)

                # 학습(역전파)
                grads = networks[key].gradient(tr_data_b, tr_lb_b)
                optimizer[key].update(networks[key].params, grads)

                # 추론(순전파)
                tr_acc, tr_loss = networks[key].acc_and_loss(tr_data_b, tr_lb_b)
                ts_acc, ts_loss = networks[key].acc_and_loss(ts_data_b, ts_lb_b)

                # 기록
                if py == 'cupy':
                    tr_acc = tr_acc.get()
                    tr_loss = tr_loss.get()
                    ts_acc = ts_acc.get()
                    ts_loss = ts_loss.get()
                train_acc[key].append(tr_acc)
                train_loss[key].append(tr_loss)
                test_acc[key].append(ts_acc)
                test_loss[key].append(ts_loss)
                end = time.time()
                time_record[key] += (end-start)

                # max accuracy 도달 해당 모델 학습 종료
                if max_acc and max_acc <= ts_acc:
                    total_step[key] = i
                    print(key + " training end!\nacc : " + str(ts_acc) + " step: " + str(i))

                # print_iter 회마다 경과 출력
                if i % print_iter == 0:
                    print("{:^9}| {:0<7.3f} | {:0<.5f} | {:0<.5f}".format(key, time_record[key], ts_acc, ts_loss))

    return train_acc, train_loss, test_acc, test_loss, total_step


def plot(label, datas, t_list, *y_lim):
    for key in t_list:
        plt.plot(smooth_curve(datas[key]), markevery=50, label=key)
    plt.xlabel("iterations")
    plt.ylabel(label)
    if y_lim:
        plt.ylim(y_lim)
    plt.legend()
    plt.show()
    
def set_data(data_name):
    if data_name == 'mnist':
        (tr_data, tr_lb), (ts_data, ts_lb) = load_mnist(normalize=True)

    elif data_name == 'mosquito':
        # DATA
        data_file = "C:/Users/dinle/Code/AI/NodeLayer/dataset/mosquito/tensor_image.pt"
        # .pt 파일을 엽니다.
        all_data = torch.load(data_file)
        all_data = np.array(all_data)
        
        # label
        lb_file = "C:/Users/dinle/Code/AI/NodeLayer/dataset/mosquito/phase2_train_v0.csv"
        df = pd.read_csv(lb_file)
        all_lb = df['class_label']
        class_num = {
            "aegypti":      0,
            "albopictus":   1,
            "anopheles":    2,
            "culex":        3,
            "culiseta":     4,
            "japonicus/koreicus": 5
        }
        all_lb = all_lb.map(class_num)
        all_lb = np.array(all_lb)
        
        # 분리
        tr_data = all_data[:2000]
        tr_lb = all_lb[:2000]
        ts_data = all_data[2000:3000]
        ts_lb = all_lb[2000:3000]
    else:
        assert 'No data'
        
    return (tr_data, tr_lb, ts_data, ts_lb)

In [27]:
# data_name = 'mnist'
data_name = 'mosquito'
batch_size = 100
max_iterations = 100
print_iter = 10
mi = np.arange(max_iterations)

In [28]:
# net1 = MLP(input_size=784, hidden_size_list=[100, 100, 100, 100], output_size=10,
#            use_dropout=False, dropout_ratio=0.5, use_batchnorm=True)
# net1 = MLP(input_size=784, hidden_size_list=[100, 100, 100, 100], output_size=10,
#             use_dropout=True, dropout_ratio=0.5, use_batchnorm=False)
# net2 = CNN(model='CNN3', use_batchnorm=True)
net3 = CNN(model='CNN8', input_dim=(3, 224, 224), output_size=6, use_batchnorm=True)
# net3 = CNN(model='VGG11', use_batchnorm=True)
# net4 = CNN(dropout_ratio=0.5, use_batchnorm=True)

In [29]:
test_list = {
             # 'MLP':
             #     {'network':net1,'opt':'AdaGrad','lr':0.01},
             # 'SCV':
             #     {'network':net2,'opt':'AdaGrad','lr':0.01},
             'CNN8':
                 {'network':net3,'opt':'AdaGrad','lr':0.01},
             # 'CNN_both':
             #     {'network':net4,'opt':'AdaGrad','lr':0.01},
             }

In [30]:
# train_acc, train_loss, test_acc, test_loss, total_step = test(test_list)
train_acc, train_loss, test_acc, test_loss, total_step = test(test_list, data_name, 0.98)

C:\Users\dinle\AppData\Local\Programs\Python\Python310\lib\site-packages\cupy\_creation\from_data.py:46: PerformanceWarning: Using synchronous transfer as pinned memory (6236073984 bytes) could not be allocated. This generally occurs because of insufficient host memory. The original error was: cudaErrorMemoryAllocation: out of memory
  return _core.array(obj, dtype, copy, order, subok, ndmin)


[Training]
cupy

[Shape]
train data: (2000, 3, 224, 224)
train label: (2000,)
test data: (1000, 3, 224, 224)
test label: (1000,)
===============iteration:0===============
  model  |  time   |   acc   |  loss   


OutOfMemoryError: Out of memory allocating 1,083,801,600 bytes (allocated so far: 13,184,034,816 bytes).

In [ ]:
plot('test_acc', test_acc , test_list)

In [ ]:
plot('test_loss', test_loss, test_list)

In [ ]:
net3.save_params("C:/Users/dinle/Code/AI/NodeLayer/params.pkl")